# Stock Estimation based on portfolio size


## Introduction

#### Leveraging IEX Cloud, yfinance and a trading bot the project emphasies on creating a day trading bot. Data provided for this project by [IEX Cloud](https://iexcloud.io), [Yahoo Finance](https://pypi.org/project/yfinance/)


#### The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of stocks you should buy.

#### Credits and inspiration from:
* #### https://github.com/nickmccullum/algorithmic-trading-python


## Library Imports

#### We need to import the open-source software libraries

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from yahoo_fin.stock_info import get_data
import yahoo_fin.stock_info as si

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Parsing API Call


In [4]:
symbol = "aapl"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Importing List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. [Yfinance](https://pypi.org/project/yfinance) was used to gather the data.

#### Let's import the stocks

In [5]:
dow_list = si.tickers_dow()
type(dow_list)

list

In [6]:
print("Tickers in Dow Jones:", len(dow_list))
dow_list[0:10]

Tickers in Dow Jones: 30


['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW']

## Making Our First API Call

Structuring our api call to IEX

We'll need the following information from the API:

* Name of the stock
* Price
* Market capitalization for each stock
* Number of shares to buy
* IEX Close
* IEX Open
* Previous Close
* Previous Volume
* Actual Close
* Close Time



In [7]:
dow_historical = {}
for ticker in dow_list:
    dow_historical[ticker] = si.get_data(ticker, start_date="01/01/2020", end_date="01/03/2020", interval="1d")
dow_historical

{'AAPL':                  open       high        low      close   adjclose     volume  \
 2020-01-02  74.059998  75.150002  73.797501  75.087502  74.444603  135480400   
 
            ticker  
 2020-01-02   AAPL  ,
 'AMGN':              open        high         low       close    adjclose   volume  \
 2020-01-02  243.0  243.190002  238.979996  240.100006  233.642029  2088000   
 
            ticker  
 2020-01-02   AMGN  ,
 'AXP':                   open        high         low       close    adjclose  \
 2020-01-02  124.660004  126.269997  124.230003  125.849998  123.637741   
 
              volume ticker  
 2020-01-02  2708000    AXP  ,
 'BA':                   open        high         low       close    adjclose  \
 2020-01-02  328.549988  333.350006  327.700012  333.320007  331.348572   
 
              volume ticker  
 2020-01-02  4544400     BA  ,
 'CAT':              open        high         low       close    adjclose   volume  \
 2020-01-02  149.0  150.550003  147.979996  150.5

## Creating pandas columns

In [8]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy', "IEX Close", "IEX Open","Previous Close","Previous Volume","Close","Close Time"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy,IEX Close,IEX Open,Previous Close,Previous Volume,Close,Close Time


## Looping Through Tickers


In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in dow_list:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([
                                                   symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A',
                                                    data['iexClose'],
                                                    data['iexOpen'],
                                                    data['previousClose'],
                                                    data['previousVolume'],
                                                    data['close'],
                                                    data['closeTime']
                                                        ],
                                                  index = my_columns), 
                                        ignore_index = True)

In [10]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy,IEX Close,IEX Open,Previous Close,Previous Volume,Close,Close Time
0,AAPL,135.370,2327667867515,N/A,132.620,135.590,139.370,148573081,None,None
1,AMGN,245.290,141650660872,N/A,248.790,251.760,255.670,5320825,None,None
2,AXP,117.040,96799616687,N/A,119.080,117.340,120.520,5860078,None,None
3,BA,202.660,114279596038,N/A,203.810,200.530,199.030,10605840,None,None
4,CAT,184.560,102554205401,N/A,187.640,184.970,192.560,3447169,None,None
5,CRM,231.610,208044228368,N/A,232.900,226.800,227.200,7262741,None,None
6,CSCO,46.490,191257076190,N/A,45.030,44.730,45.940,21654297,None,None
7,CVX,85.800,165681739019,N/A,86.820,85.520,89.810,8251947,None,None
8,DIS,174.710,305306362362,N/A,175.240,176.160,177.760,15010740,None,None
9,DOW,51.900,38987908871,N/A,54.150,53.840,54.780,5954607,None,None


In [11]:
final_dataframe.shape

(30, 10)

## Using Batch API Calls to Improve Performance

We will split our list of stocks into groups of 5

In [12]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [13]:
symbol_groups = list(chunks(final_dataframe['Ticker'], 5))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])




AAPL,AMGN,AXP,BA,CAT
CRM,CSCO,CVX,DIS,DOW
GS,HD,HON,IBM,INTC
JNJ,JPM,KO,MCD,MMM
MRK,MSFT,NKE,PG,TRV
UNH,V,VZ,WBA,WMT


In [14]:

final_dataframe_2 = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data_b = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe_2 = final_dataframe.append(
                                        pd.Series([
                                                   symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A',
                                                    data['iexClose'],
                                                    data['iexOpen'],
                                                    data['previousClose'],
                                                    data['previousVolume'],
                                                    data['close'],
                                                    data['closeTime']
                                                        ],
                                                  index = my_columns), 
                                        ignore_index = True)
        

## Calculating the Number of Shares to Buy

In [15]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [16]:
position_size = float(portfolio_size) / len(final_dataframe_2.index)
for i in range(0, len(final_dataframe_2['Ticker'])-1):
    final_dataframe_2.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe_2['Price'][i])
print(math.floor(position_size))
final_dataframe_2.head(10)

32258


,Ticker,Price,Market Capitalization,Number Of Shares to Buy,IEX Close,IEX Open,Previous Close,Previous Volume,Close,Close Time
0,AAPL,135.37,2327667867515,238,132.62,135.59,139.37,148573081,None,None
1,AMGN,245.29,141650660872,131,248.79,251.76,255.67,5320825,None,None
2,AXP,117.04,96799616687,275,119.08,117.34,120.52,5860078,None,None
3,BA,202.66,114279596038,159,203.81,200.53,199.03,10605840,None,None
4,CAT,184.56,102554205401,174,187.64,184.97,192.56,3447169,None,None
5,CRM,231.61,208044228368,139,232.90,226.80,227.20,7262741,None,None
6,CSCO,46.49,191257076190,693,45.03,44.73,45.94,21654297,None,None
7,CVX,85.80,165681739019,375,86.82,85.52,89.81,8251947,None,None
8,DIS,174.71,305306362362,184,175.24,176.16,177.76,15010740,None,None
9,DOW,51.90,38987908871,621,54.15,53.84,54.78,5954607,None,None


### Number of shares to buy for each stock

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [17]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe_2.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [19]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format],
                    'E': ['IEX Close' , integer_format],
                    'F': ['IEX Open', integer_format],
                    'G': ['Previous Close', integer_format],
                    'H': ['Previous Volume', integer_format],
                    'I': ['Actual Close', integer_format],
                    'J': ['Close Time', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [20]:
writer.save()